In [ ]:
import datetime
import panel as pn

pn.extension('tabulator', notifications=True)

import panel as pn
import matplotlib.pyplot as plt
from src.dao import pessoa_dao, plantio_dao, propriedade_dao

pn.extension('tabulator', notifications=True)

# Título do Sistema
titulo = pn.pane.Markdown("# 🚜 AgroGestão - Sistema Integrado")

In [ ]:
import panel as pn
import matplotlib.pyplot as plt
from src.dao import pessoa_dao # Importa o novo DAO

# --- WIDGETS ---
w_pes_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)
w_pes_nome = pn.widgets.TextInput(name="Nome Completo")
w_pes_cpf = pn.widgets.TextInput(name="CPF")
w_pes_nasc = pn.widgets.DatePicker(name="Data Nascimento")

# Checkboxes para definir o papel
w_check_prod = pn.widgets.Checkbox(name="É Produtor?", value=True)
w_check_tec = pn.widgets.Checkbox(name="É Técnico?", value=False)

# Campos Específicos (inicialmente visíveis ou não baseados no check)
w_pes_insc = pn.widgets.TextInput(name="Inscrição Estadual (Produtor)")
w_pes_crea = pn.widgets.TextInput(name="CREA (Técnico)")

# Lógica de Visibilidade (Interatividade Pura)
def toggle_produtor(event):
    w_pes_insc.visible = event.new
    w_pes_insc.disabled = not event.new

def toggle_tecnico(event):
    w_pes_crea.visible = event.new
    w_pes_crea.disabled = not event.new

# Liga os eventos
w_check_prod.param.watch(toggle_produtor, 'value')
w_check_tec.param.watch(toggle_tecnico, 'value')

# Inicializa estado
w_pes_insc.visible = True
w_pes_crea.visible = False

# Botões e Tabela
btn_pes_salvar = pn.widgets.Button(name='💾 Salvar Pessoa', button_type='success')
btn_pes_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_pes_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_pes = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_pes = pn.pane.Matplotlib(tight=True)

# --- LÓGICA DE DADOS ---

async def carregar_pessoas(event=None):
    tab_pes.value = await pessoa_dao.listar()
    
    # Gráfico de Pizza (Produtores vs Técnicos)
    df_g = await pessoa_dao.dados_grafico_papeis()
    if not df_g.empty:
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.bar(df_g['Tipo'], df_g['Total'], color=['#4CAF50', '#2196F3'])
        ax.set_title("Distribuição de Cadastros")
        plt.close(fig)
        graf_pes.object = fig

def selecionar_pessoa(event):
    if event.row is None: return
    try:
        row_index = event.row
        df = tab_pes.value
        row = df.iloc[row_index]

        w_pes_id.value = int(row['id'])
        w_pes_nome.value = str(row['Nome'])
        w_pes_cpf.value = str(row['CPF'])
        
        # Data
        if row['Nascimento']:
            val = row['Nascimento']
            w_pes_nasc.value = val.date() if hasattr(val, 'date') else val
            
        # Papéis (Define checkboxes)
        is_prod = bool(row['É Produtor?'])
        is_tec = bool(row['É Técnico?'])
        
        w_check_prod.value = is_prod
        w_check_tec.value = is_tec
        
        # Preenche campos específicos se existirem
        if is_prod: w_pes_insc.value = str(row['Inscrição Est.'])
        else: w_pes_insc.value = ""
            
        if is_tec: w_pes_crea.value = str(row['CREA']) if row['CREA'] else ""
        else: w_pes_crea.value = ""

        btn_pes_salvar.name = "🔄 Atualizar"
        btn_pes_salvar.button_type = "warning"
        btn_pes_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando: {row['Nome']}")
    except Exception as e:
        print(f"Erro: {e}")

async def salvar_pessoa(event):
    try:
        if not w_pes_nome.value or not w_pes_cpf.value:
            pn.state.notifications.error("Nome e CPF obrigatórios!")
            return
            
        if not w_check_prod.value and not w_check_tec.value:
             pn.state.notifications.warning("A pessoa deve ser Produtor, Técnico ou Ambos!")
             return

        btn_pes_salvar.loading = True
        
        await pessoa_dao.salvar(
            w_pes_id.value if w_pes_id.value > 0 else None,
            w_pes_nome.value, w_pes_cpf.value, w_pes_nasc.value,
            w_check_prod.value, w_pes_insc.value,
            w_check_tec.value, w_pes_crea.value
        )
        
        pn.state.notifications.success("Salvo com sucesso!")
        limpar_pessoa(None)
        await carregar_pessoas()
        
    except Exception as e:
        pn.state.notifications.error(f"Erro: {e}")
    finally:
        btn_pes_salvar.loading = False

async def excluir_pessoa(event):
    if w_pes_id.value:
        await pessoa_dao.excluir(w_pes_id.value)
        pn.state.notifications.success("Pessoa excluída!")
        limpar_pessoa(None)
        await carregar_pessoas()

def limpar_pessoa(event):
    w_pes_id.value = 0
    w_pes_nome.value = ""
    w_pes_cpf.value = ""
    w_pes_insc.value = ""
    w_pes_crea.value = ""
    w_check_prod.value = True
    w_check_tec.value = False
    btn_pes_salvar.name = "💾 Salvar Pessoa"
    btn_pes_salvar.button_type = "success"
    btn_pes_excluir.disabled = True

# --- LIGAÇÕES ---
btn_pes_salvar.on_click(salvar_pessoa)
btn_pes_limpar.on_click(limpar_pessoa)
btn_pes_excluir.on_click(excluir_pessoa)
tab_pes.on_click(selecionar_pessoa)

# --- LAYOUT ---
layout_produtor = pn.Row(
    pn.Column(
        "### Dados Pessoais", 
        w_pes_nome, w_pes_cpf, w_pes_nasc,
        "### Papéis",
        pn.Row(w_check_prod, w_check_tec),
        w_pes_insc, w_pes_crea,
        pn.Row(btn_pes_salvar, btn_pes_limpar, btn_pes_excluir)
    ),
    pn.Column(
        "### Base de Pessoas", 
        tab_pes, 
        "### Estatísticas",
        graf_pes
    )
)

# Carrega ao iniciar
pn.state.onload(carregar_pessoas)

In [ ]:
# --- WIDGETS ---
w_plant_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

# Selects (Carregados do Banco)
w_sel_safra = pn.widgets.Select(name="Safra")
w_sel_cultivo = pn.widgets.Select(name="Cultivo/Variedade")
w_sel_prop = pn.widgets.Select(name="Propriedade")

w_data_plantio = pn.widgets.DatePicker(name="Data do Plantio", value=datetime.date.today())
w_area_plantada = pn.widgets.FloatInput(name="Área Plantada (Hectares)", value=0.0, step=0.1)

btn_plant_salvar = pn.widgets.Button(name='🌱 Registrar Plantio', button_type='success')
btn_plant_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_plant_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_plant = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_plant = pn.pane.Matplotlib(tight=True)

# --- LÓGICA ---

async def carregar_opcoes_plantio():
    """Preenche os menus com dados do banco"""
    w_sel_safra.options = await plantio_dao.get_opcoes_safra()
    w_sel_cultivo.options = await plantio_dao.get_opcoes_cultivo()
    w_sel_prop.options = await plantio_dao.get_opcoes_propriedade()

async def carregar_plantios(event=None):
    # 1. Carrega opções (para garantir que estão atualizadas)
    await carregar_opcoes_plantio()
    
    # 2. Carrega Tabela
    tab_plant.value = await plantio_dao.listar()
    
    # 3. Carrega Gráfico
    df_g = await plantio_dao.dados_grafico_cultura()
    if not df_g.empty:
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.pie(df_g['Total Hectares'], labels=df_g['Cultura'], autopct='%1.1f%%', startangle=90, colors=['#8dd3c7','#ffffb3','#bebada','#fb8072'])
        ax.set_title("Área Plantada por Cultura")
        plt.close(fig)
        graf_plant.object = fig

def selecionar_plantio(event):
    # 1. Verificação de segurança
    if event.row is None:
        return

    try:
        # CORREÇÃO: Usar o índice para buscar os dados no DataFrame da tabela
        row_index = event.row
        df = tab_plant.value # Pega o DataFrame atual da tabela
        row = df.iloc[row_index] # Acessa a linha correta
        
        print(f"✅ Dados Plantio selecionados: {row.to_dict()}")

        # 2. Preencher os Campos (Baseado nos nomes das colunas do plantio_dao)
        # Colunas esperadas: 'id', 'Safra', 'Cultura', 'Propriedade', 'Data Plantio', 'Área (ha)'
        
        w_plant_id.value = int(row['id'])
        
        # Área (tem de converter para float para garantir)
        if 'Área (ha)' in row:
            w_area_plantada.value = float(row['Área (ha)'])
            
        # Data
        if 'Data Plantio' in row and row['Data Plantio']:
            val = row['Data Plantio']
            if hasattr(val, 'date'):
                w_data_plantio.value = val.date()
            else:
                w_data_plantio.value = val # Tenta atribuir direto se já for date
        
        # 3. Atualizar Estado dos Botões
        btn_plant_salvar.name = "🔄 Atualizar Dados"
        btn_plant_salvar.button_type = "warning"
        btn_plant_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando Plantio #{row['id']}")

    except Exception as e:
        msg = f"❌ Erro ao selecionar plantio: {str(e)}"
        print(msg)
        pn.state.notifications.error(msg)

async def salvar_plantio(event):
    try:
        if not w_sel_safra.value or not w_sel_cultivo.value or not w_sel_prop.value:
            pn.state.notifications.error("Selecione Safra, Cultivo e Propriedade!")
            return
        if w_area_plantada.value <= 0:
            pn.state.notifications.error("A área deve ser maior que zero!")
            return

        btn_plant_salvar.loading = True
        
        if w_plant_id.value: # Atualizar
            await plantio_dao.atualizar(
                w_plant_id.value, w_sel_safra.value, w_sel_cultivo.value, 
                w_sel_prop.value, w_data_plantio.value, w_area_plantada.value
            )
            pn.state.notifications.success("Plantio atualizado!")
        else: # Inserir
            await plantio_dao.inserir(
                w_sel_safra.value, w_sel_cultivo.value, 
                w_sel_prop.value, w_data_plantio.value, w_area_plantada.value
            )
            pn.state.notifications.success("Plantio registrado!")

        limpar_plantio(None)
        await carregar_plantios()
    except Exception as e:
        pn.state.notifications.error(f"Erro: {e}")
    finally:
        btn_plant_salvar.loading = False

async def excluir_plantio(event):
    if w_plant_id.value:
        try:
            await plantio_dao.excluir(w_plant_id.value)
            pn.state.notifications.success("Plantio excluído!")
            limpar_plantio(None)
            await carregar_plantios()
        except Exception as e:
            pn.state.notifications.error(f"Erro ao excluir: {e}")

def limpar_plantio(event):
    w_plant_id.value = 0
    w_data_plantio.value = datetime.date.today()
    w_area_plantada.value = 0.0
    btn_plant_salvar.name = "🌱 Registrar Plantio"
    btn_plant_salvar.button_type = "success"
    btn_plant_excluir.disabled = True

# --- LIGAÇÕES ---
btn_plant_salvar.on_click(salvar_plantio)
btn_plant_limpar.on_click(limpar_plantio)
btn_plant_excluir.on_click(excluir_plantio)
tab_plant.on_click(selecionar_plantio)

# --- LAYOUT ---
layout_plantio = pn.Row(
    pn.Column(
        "### 🚜 Novo Plantio", 
        w_sel_safra, 
        w_sel_cultivo, 
        w_sel_prop, 
        w_data_plantio, 
        w_area_plantada, 
        pn.Row(btn_plant_salvar, btn_plant_limpar, btn_plant_excluir)
    ),
    pn.Column(
        "### Histórico de Plantios", 
        tab_plant, 
        "### Estatísticas (Área)",
        graf_plant
    )
)

In [ ]:
# CÉLULA 5 - DASHBOARD FINAL

# Função para carregar tudo ao abrir
async def inicializar_tudo():
    await carregar_produtores()
#    await carregar_propriedades()
    await carregar_plantios()

pn.state.onload(inicializar_tudo)

# Cria as Abas Navegáveis
dashboard = pn.Tabs(
    ("🧑‍🌾 PPessoas", layout_produtor),
 #   ("🏡 Propriedades", layout_propriedade),
    ("🌱 Plantios", layout_plantio)
)

# Exibe o Dashboard Completo
pn.Column(
    titulo,
    dashboard
).servable()